In [ ]:
import cv2
import time
import os
import glob
import json
import numpy as np
import natsort
import pickle5 as pickle
import matplotlib.pyplot as plt
from multiprocessing import Process
from pathlib import Path
import glob
import faiss
from models.vit.main import main as ret_vit

# Validation
    1. Parsing
    2. Matching& Validation

## 1. Parsing Data for validation
    - json example
    {
      "version": "5.1.1",
      "flags": {},
      "shapes": [
        {
          "label": "1",
          "points": [
            [
              234.0,
              362.0
            ],
            [
              251.0,
              362.0
            ],
            [
              251.0,
              426.0
            ],
            [
              234.0,
              426.0
            ]
          ],
          "group_id": null,
          "shape_type": "polygon",
          "flags": {
            "matched": true,
            "changed": false
          }
        },
    - {cropped_index}_{label}_{changed_flag}
        ex) 0_100_False.jpg -> label 100 is unmatched pair
            1_3_False.jpg -> False means unchanged design signs

In [11]:
# unlbl_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_ori/query/'
# save_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/db_crop_val/'

def parsing(img_lbl_dir, save_dir):
    
    '''
    img_lbl_dir : str, ***the directory has image & label json of same file name ex) 400.jpg/ 400.json
    save_dir : str, the directory of saving parsing image files
    '''

    for i, (img_path, lbl_path) in enumerate(zip(sorted(glob.glob(img_lbl_dir + '*.jpg')), sorted(glob.glob(img_lbl_dir + '*.json')))): #######

        #process
        print(f"{i}/{len(img_path)}")

        #check match img with label
        img_id = int(Path(img_path).stem.split('@')[0])
        lbl_id = int(Path(lbl_path).stem.split('@')[0])
        if img_id == lbl_id:
            print(f" img_id : {img_id}, label_id : {lbl_id}")

            #read image origin
            img_org = cv2.imread(img_path)

            #open label
            with open(lbl_path, "r") as js:
                lbl = json.load(js)

                #read label
                for i in range(len(lbl['shapes'])):

                    # crop image
                    coord_list = lbl['shapes'][i]['points']
                    x_list = list(map(lambda x: int(x[0]), coord_list))
                    y_list = list(map(lambda x: int(x[1]), coord_list))
                    del coord_list

                    min_x = min(x_list)
                    max_x = max(x_list)
                    del x_list

                    min_y = min(y_list)
                    max_y = max(y_list)
                    del y_list

                    croppedImage = img_org[min_y: max_y, min_x: max_x]

                    # match info
                    match = lbl['shapes'][i]['label'] #if matched specific id, unmatched 100
                    change = str(lbl['shapes'][i]['flags']['changed']) # False F, True T

                    #save cropped Image
                    dir_cropped = save_dir + f"/{img_id}/" #####
                    if not os.path.exists(dir_cropped):
                        os.makedirs(dir_cropped, exist_ok=True)
                    cropped_fname = f"{str(i)}_{match}_{change}"
                    cv2.imwrite(dir_cropped + cropped_fname + '.jpg', croppedImage)
        else:
            raise.Exception("ID Matching Error")

In [12]:
### exec
img_lbl_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_ori/query/'
save_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/test_val/'

parsing(img_lbl_dir, save_dir)

0/96
 img_id : 0, label_id : 0
1/98
 img_id : 100, label_id : 100
2/97
 img_id : 10, label_id : 10
3/98
 img_id : 110, label_id : 110
4/98
 img_id : 120, label_id : 120
5/98
 img_id : 130, label_id : 130
6/98
 img_id : 140, label_id : 140
7/98
 img_id : 150, label_id : 150
8/98
 img_id : 160, label_id : 160
9/98
 img_id : 170, label_id : 170
10/98
 img_id : 180, label_id : 180
11/98
 img_id : 190, label_id : 190
12/98
 img_id : 200, label_id : 200
13/97
 img_id : 20, label_id : 20
14/98
 img_id : 210, label_id : 210
15/98
 img_id : 220, label_id : 220
16/98
 img_id : 230, label_id : 230
17/98
 img_id : 240, label_id : 240
18/98
 img_id : 250, label_id : 250
19/98
 img_id : 260, label_id : 260
20/98
 img_id : 270, label_id : 270
21/98
 img_id : 280, label_id : 280
22/98
 img_id : 290, label_id : 290
23/98
 img_id : 300, label_id : 300
24/97
 img_id : 30, label_id : 30
25/98
 img_id : 310, label_id : 310
26/98
 img_id : 320, label_id : 320
27/98
 img_id : 330, label_id : 330
28/98
 img_i

## 2. Matching& Validation

In [8]:
#### Path
!rm -rf ./panorama_test/panorama_crop/q_crop_val/.ipynb_checkpoints/
!rm -rf ./panorama_test/panorama_crop/db_crop_val/.ipynb_checkpoints/

# DB dataset root path
db_root = # ./db_val/

pnorm_db_dic = {} #panorama db 
for db in os.listdir(db_root):
    
    if db != 'labels':
        db_root = db_root + db + '/'
        db_path_list = natsort.natsorted([db_root + db_name for db_name in os.listdir(db_root) if 'jpg' in db_name or 'png' in db_name])
        pnorm_db_dic[db] = db_path_list
        
# Query dataset root path        
q_root = #./q_val/

pnorm_q_dic = {} #panorama query
for q in os.listdir(q_root):
    
    if q != 'labels':
        q_root = pnorm_q_root + q + '/'
        q_path_list =natsort.natsorted( [q_root + q_name for q_name in os.listdir(q_root) if 'jpg' in q_name or 'png' in q_name])
        pnorm_q_dic[q] = q_path_list

### 2.1 Feature Descriptor
    1. SIFT
    2. RootSIFT

In [9]:
# SIFT
def sift_extractor(file_path):
    '''
    Description: extract \emph{sift} feature from given image
    Input: file_path - image path
    Output: des - a list of descriptors of all the keypoint from the image
    '''
    img = cv2.imread(file_path)
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create() # 1000
    _,des = sift.detectAndCompute(gray,None) 

    return des

In [10]:
# RootSIFT
# import the necessary packages

class RootSIFT:
	def __init__(self, max_kps):
		# initialize the SIFT feature extractor
		self.max_kps = max_kps
		self.extractor = cv2.SIFT_create(self.max_kps)
	def compute(self, image, kps, eps=1e-7):
		# compute SIFT descriptors
		kps, descs = self.extractor.detectAndCompute(image,kps)
		# if there are no keypoints or descriptors, return an empty tuple
		if len(kps) == 0:
			return ([], None)
		# apply the Hellinger kernel by first L1-normalizing and taking the
		# square-root(L2-noramlized)
		descs /= (descs.sum(axis=1, keepdims=True) + eps)
		descs = np.sqrt(descs)
		#descs /= (np.linalg.norm(descs, axis=1, ord=2) + eps)
		# return a tuple of the keypoints and descriptors
		return (kps, descs)

In [11]:
# RootSIFT
# input: file_path
def rootsift_extractor(file_path):
    '''
    Description: extract \emph{sift} feature from given image
    Input: file_path - image path
    Output: des - a list of descriptors of all the keypoint from the image
    '''
    img = cv2.imread(file_path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    rs = RootSIFT(99999) #full
    _, des = rs.compute(gray, None) 

    return des

### - utils
    1. k-means clustering
    2. get vlad vector
    3. L2 distance(numpy)

In [12]:
def get_cluster_center(des_set, K):
    '''
    Description: cluter using a default setting
    Input: des_set - cluster data
                 K - the number of cluster center
    Output: laber  - a np array of the nearest center for each cluster data
            center - a np array of the K cluster center
    '''
    des_set = np.float32(des_set) # for kmeans 
    
##########################[CPU-SLOW OPENCV KMEANS]##########################
    # criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
    
    # ret, label, center = cv2.kmeans(des_set, K, None, criteria, 3, cv2.KMEANS_RANDOM_CENTERS)
#########################################################################

##########################[GPU-FAST FAISS KMEANS]##########################
    # Setup
    kmeans = faiss.Kmeans(d=128, k=K, niter=20, verbose=True, gpu=True)
    
    # Run clustering
    kmeans.train(des_set)
    
    # Centroids after clustering
    center = kmeans.centroids
    
    # The assignment for each vector.
    _, label = kmeans.index.search(des_set, 1)  # Need to run NN search again
    
    return label, center

In [13]:
def get_des_vector(file_path_list, method='rootsift', DESDIM=128):
    '''
    Description: get descriptors of all the images 
    Input: file_path_list - all images path
           method - str, default = 'rootsift', the way of extracting feature ['sift', 'rootsift', 'orb']
    Output:       all_des - a np array of all descriptors
            image_des_len - a list of number of the keypoints for each image 
    '''
    
    # all_des = np.empty(shape=[0, DESDIM]) #float64
    all_des = np.float32([]).reshape(0,DESDIM) #float32
    image_des_len = []

    for eachFile in file_path_list:
        
        try:
            des = eval(f"{method}_extractor('{eachFile}')") # RootSIFT                
            all_des = np.concatenate([all_des, des]) #모든 이미지의 des vector를 concat 해서 출력! #np.concat : 다차원 배열의 결과
            image_des_len.append(len(des)) #각각 이미지의 len(des)을 따로 list에 모아두기.
        except:
            print(eachFile)
            print("extract feature error")
    
    # feature num count
    featCnt = all_des.shape[0] #kp 개수
    print(str(featCnt) + " features in " + " images")
    
    return all_des, image_des_len, featCnt

In [14]:
def get_codebook(all_des, K):
    '''
    Description: train the codebook from all of the descriptors
    Input: all_des - training data for the codebook
                 K - the column of the codebook
    '''
    label, center = get_cluster_center(all_des, K)
    return label, center #center -> codebook

In [15]:
def get_vlad_base(img_des_len, NNlabel, all_des, codebook, COLUMNOFCODEBOOK, DESDIM=128):
    '''
    Description: get all images vlad vector 
    '''
    cursor = 0
    vlad_base = []
    for i, eachImage in enumerate(img_des_len):
        
        descrips = all_des[cursor : cursor + eachImage]
        centriods_id = NNlabel[cursor : cursor + eachImage]
        centriods = codebook[centriods_id]
    
        vlad = np.zeros(shape=[COLUMNOFCODEBOOK, DESDIM]).astype(np.float32) # VLAD vector: K * D 차원
        for eachDes in range(eachImage):
            vlad[centriods_id[eachDes]] = vlad[centriods_id[eachDes]] + descrips[eachDes] - centriods[eachDes] #차원 별 각각 중심으로부터 거리 (lacal feature 값 - NN 중심 값)
        cursor += eachImage # cursor 이동
    
        vlad_norm = vlad.copy()
        cv2.normalize(vlad, vlad_norm, 1.0, 0.0, cv2.NORM_L2) # 0에서 1사이로 L2 Norm
        vlad_base.append(vlad_norm.reshape(COLUMNOFCODEBOOK * DESDIM, -1)) # vlad_norm : 이미지 1장 -> vlad_base : DB 모든 이미지 M장 = (K*D)*M
    
    print("get_db_vlad_feat")
    
    return vlad_base #VLAD Base Vector

In [16]:
def cal_vec_dist(vec1, vec2):
    '''
    Description: calculate the Euclidean Distance of two vectors (norm_l2)
    '''
    return np.linalg.norm(vec1 - vec2)

# query에 대해서 ON_LINE에서 진행되는 과정
def get_pic_vlad(pic, des_size, codebook, COLUMNOFCODEBOOK, DESDIM=128): # query의 vlad vector 만들기 by db의 codebook으로!
    '''
    Description: get the vlad vector of each image
    '''
    vlad = np.zeros(shape=[COLUMNOFCODEBOOK, DESDIM])
    for eachDes in range(des_size):
        des = pic[eachDes]
        min_dist = 1000000000.0
        ind = 0
        for i in range(COLUMNOFCODEBOOK):
            dist = cal_vec_dist(des, codebook[i]) # 각각 local feature가 중심으로부터 떨어진 거리
            if dist < min_dist: # 최근접 중심
                min_dist = dist
                ind = i
        vlad[ind] = vlad[ind] + des - codebook[ind] # 최근접 중심 + 떨어진 거리 : Vlad 정의
    
    vlad_norm = vlad.copy()
    cv2.normalize(vlad, vlad_norm, 1.0, 0.0, cv2.NORM_L2) #L2 Norm
    vlad_norm = vlad_norm.reshape(COLUMNOFCODEBOOK * DESDIM, -1)
    
    return vlad_norm

### 2.2 DB : get_db
    - In offline, we make db feat in advance and save it.

In [133]:
def get_db(db_path_list, save_path, featset):

    #time check - Time of learning codebook(Off_line)
    cb_t_b = time.perf_counter()

    ##get all the descriptor vectors of the data set
    print("db_feature_num")
    all_des, image_des_len, _ = get_des_vector(db_path_list, method='rootsift')

    # Time of learning codebook
    cb_t = time.perf_counter() - cb_t_b

    print(f"Time of Learning Codebook(Off_line):{cb_t}")

    #############save############
    # save all descriptors to file
    
    # save all_des
    os.makedirs(save_path, exist_ok=True)
    with open(save_path + f"test_db_{featset}_rs" + ".pkl" , 'wb') as fp: #wb, save
        pickle.dump(all_des, fp, protocol=pickle.HIGHEST_PROTOCOL)
    del all_des #save memory

    # save image_des_len(list)
    os.makedirs(save_path, exist_ok=True)
    with open(save_path + f"test_db_vlad_{featset}_len" + ".pkl" , 'wb') as fp: #wb, save
        pickle.dump(image_des_len, fp, protocol=pickle.HIGHEST_PROTOCOL)
    del image_des_len #save memory
    
    # alarm
    print(f"get_db_{featset}")
###exec####

# DESDIM = 128 # SIFT - 128 / ORB - 32

save_path = "/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/checkpoint/feat_set/"
#pnorm_db_dic[db] = db_path_list

start = time.perf_counter()

for i, db in enumerate(pnorm_db_dic.keys()):
    
    #process
    print(f"{i}/{len(pnorm_db_dic)}")
    
    #exec
    get_db(db_path_list=pnorm_db_dic[db], save_path=save_path, featset=db)

get_db_time = time.perf_counter() - start
print(f"get_db_time : {get_db_time}", f"get_db_per_query : {get_db_time/len(pnorm_db_dic)}")

#time
# get_db_time : 2.7983252509729937 get_db_per_query : 0.028848713927556637
# get_db_time : 3.2516509029082954 get_db_per_query : 0.03352217425678655

0/97
db_feature_num
2419 features in  images
Time of Learning Codebook(Off_line):0.08417913306038827
get_db_870
1/97
db_feature_num
414 features in  images
Time of Learning Codebook(Off_line):0.016131491982378066
get_db_150
2/97
db_feature_num
468 features in  images
Time of Learning Codebook(Off_line):0.015606119064614177
get_db_930
3/97
db_feature_num
168 features in  images
Time of Learning Codebook(Off_line):0.009488218114711344
get_db_580
4/97
db_feature_num
912 features in  images
Time of Learning Codebook(Off_line):0.02896014496218413
get_db_370
5/97
db_feature_num
1006 features in  images
Time of Learning Codebook(Off_line):0.029236466041766107
get_db_730
6/97
db_feature_num
124 features in  images
Time of Learning Codebook(Off_line):0.0065057630417868495
get_db_160
7/97
db_feature_num
334 features in  images
Time of Learning Codebook(Off_line):0.025386565015651286
get_db_290
8/97
db_feature_num
813 features in  images
Time of Learning Codebook(Off_line):0.03371242305729538
get

### 2.3 Retrieval : ret_vlad
    - SIFT Descriptor & VLAD feature aggregation
    - faiss, Cosine Similarity Matching

In [137]:
# panorama 

# def ret_kp(q_path_list, db_path_list, panorama_id, dic_save_path, txt_save_path, matcher='bf', metric="l2", method='rootsift', \
#            val = "union", cross_check=True, img_format='jpg'):

def ret_vlad(q_path_list, db_path_list, load_path, panorama_id, dic_save_path, txt_save_path,method=='rootsift',\
             metric="cs", device='cpu'): # for iter
    
    '''
    method : str, how to extract feature from image, ['rootsift', 'sift']
    metric : str, how to compare with vectors, ["l2", "cs"], l2 is Euclidean Distance, cs is Cosine Similarity
    device : str, if 'cpu', faiss search with cpu else, 'gpu' or 'cuda', anything.
    '''
    
    # load DB all descriptors
    fp = load_path + f"test_db_{panorama_id}_rs" + ".pkl" 
    all_des = pickle.load(open(fp, "rb"))

    # load image_des_len(list)
    with open(load_path + f'test_db_vlad_{panorama_id}_len' + '.pkl', 'rb') as fp: #len은 동일
        image_des_len = pickle.load(fp)

    #time check - Time of Retrieval(On_line)
    ret_t_b = time.perf_counter()

    ##get all the descriptor vectors of the query set
    print("q_feature_num")
    ret_des, ret_des_len, featCnt = get_des_vector(q_path_list, method)
    
    #faiss clustering auto setting
    COLUMNOFCODEBOOK = int(featCnt / 39)

    ##trainning the codebook
    NNlabel, codebook = get_codebook(all_des, COLUMNOFCODEBOOK) #DB 사이즈에 따라 CD 크기가 바뀌므로, 매번 뽑아줘야 한다.
    
    #get all vlad vectors of the db set
    vlad_base = get_vlad_base(image_des_len, NNlabel, all_des, codebook, COLUMNOFCODEBOOK)
    del all_des #save memory
    
    db_num = len(image_des_len)
    del image_des_len #save memory

    ##get all the vlad vectors of retrival set without pca dimensionality reduction
    cursor_ret = 0
    ret_vlad_list = []
    for i, eachretpic in enumerate(range(len(ret_des_len))): #각각 이미지 des 길이들 만큼만 pick! -> 이미지 1장 1장에 대해 retrieval 해야 하므로!
        print(f"{i}/{len(ret_des_len)}")
        pic = ret_des[cursor_ret: cursor_ret + ret_des_len[eachretpic]]
        ret_vlad = get_pic_vlad(pic, ret_des_len[eachretpic], codebook, COLUMNOFCODEBOOK) # COLUMNOFCODEBOOK #CD
        cursor_ret += ret_des_len[eachretpic] # next cursor
        ret_vlad_list.append(ret_vlad)
    del ret_des, ret_des_len #save memory
    
# ==
    ### for faiss ###
    ### kernel died because of doulbing memory by copying it
    vlad_base = np.array(vlad_base).reshape(len(vlad_base), -1) #(개수, 차원) ex) K=5500 (3369, 704000)
    vlad_base = vlad_base.astype(np.float32)
    
    ret_vlad_set = np.array(ret_vlad_list).reshape(len(ret_vlad_list), -1) #(개수, 차원) ex) K=5500, (100, 704,000)
    ret_vlad_set = ret_vlad_set.astype(np.float32)
    
    print("faiss_search")
    start_rd = time.perf_counter()
    
    ## faiss similarity search ##
    faiss.normalize_L2(vlad_base) # 자신의 절댓값 크기(sqrt(self**2))로 나눠준다.
    faiss.normalize_L2(ret_vlad_set)
    
    dim = COLUMNOFCODEBOOK * DESDIM # vlad_dim: K*D
    if metric == "l2":  # Norm L2 Dist
        cpu_index = faiss.IndexFlatL2(dim)
        
    else : # cosine similarity
        cpu_index = faiss.IndexFlatIP(dim)
        
    if device == 'cpu':
        cpu_index.add(vlad_base)
        del vlad_base #save memory

        start = time.perf_counter()
        D, I = cpu_index.search(ret_vlad_set, cpu_index.ntotal) # 전체 쿼리(병렬) 몇 등까지 볼거야? 전체 다!
        print(' faiss search time: ', time.perf_counter()-start)
        
    else: #'cuda', 'gpu'
        #Clone from CPU to GPU
        co = faiss.GpuMultipleClonerOptions()
        co.shard = True
        co.useFloat16 = True
        index = faiss.index_cpu_to_all_gpus(cpu_index, co=co) # build the index
        index.add(vlad_base)
        del vlad_base #save memory
        
        start = time.perf_counter()
        D, I = index.search(ret_vlad_set, index.ntotal) # 전체 쿼리(병렬) 몇 등까지 볼거야? 전체 다!
        print(' faiss search time: ', time.perf_counter()-start)
        
    q_num = ret_vlad_set.shape[0]
    del ret_vlad_set #save memory
    
    #out result dic
    result_dic = {}
    
    for q_idx in range(q_num): # query index
        eval_list = []
        for db_idx in range(db_num): # db index
            eval_list.append((I[q_idx,db_idx], D[q_idx,db_idx]))
        result_dic[q_idx] = eval_list
    
    # get_result_time
    print("get_result_dic", time.perf_counter()-start_rd)

    #time check - Time of Retrieval(On_line)
    ret_t = time.perf_counter() - ret_t_b
    print(f"Time of Retrieval(On_line):{ret_t}")
    
   #save result dict
    os.makedirs(dic_save_path + f"/{metric}/", exist_ok=True)
    with open(dic_save_path + f"/{metric}/dic_{panorama_id}_sift" + ".p", 'wb') as fp: #rs : RootSIFT
        pickle.dump(result_dic, fp, protocol=pickle.HIGHEST_PROTOCOL)

    # save top1 pair .txt
    os.makedirs(txt_save_path + f"/{metric}/", exist_ok=True)
    out_file = txt_save_path + f"/{metric}/pair_{panorama_id}_sift" + ".txt"
    
    # if repeated, the lines is accumulated
    if os.path.isfile(out_file):
        os.remove(out_file)
    
    f_out = open(out_file, 'w')

    for q_idx in list(result_dic.keys()):
        #### L2 Distance/ Cos_Sim ####
        if metric == "l2":  # Norm L2 Dist
            rank_list = sorted(result_dic[q_idx], key = lambda x: x[1]) # Norm l2 dist
        else : # cosine similarity
            rank_list = sorted(result_dic[q_idx], key = lambda x: -x[1]) #cos sim
        ##############################
        for (db_idx, score)  in rank_list:
            query_name = Path(q_path_list[q_idx]).stem
            db_name = Path(db_path_list[db_idx]).stem
            
            f_out.write(query_name + '.jpg,' + db_name + '.jpg,' + str(score) + '\n')
            f_out.flush()
    f_out.close()
     
### exec ###

load_path = "/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/checkpoint/feat_set/"
dic_save_path = "/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/result_dic/vlad/"
txt_save_path = "/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/pair_txt/vlad/"

start = time.perf_counter()
for i, q in enumerate(pnorm_q_dic.keys()):

    #process
    print(f"{i}/{len(pnorm_q_dic)}")
    
    # try: # query ID == db ID
        #exec
    ret_vlad(q_path_list=pnorm_q_dic[q], db_path_list=pnorm_db_dic[q], panorama_id=q, \

             load_path=load_path, dic_save_path=dic_save_path, txt_save_path=txt_save_path, metric="cs")

    # except: # if query ID not in db ID
    #     print("ID error")
    #     print(f"query ID : {q}")
        
print("total_ret_time", time.perf_counter()-start)
#time
# total_ret_time 40.83246005396359
# ret_time_per_q 0.4209531964326143

0/97
q_feature_num
2312 features in  images

Clustering 2419 points in 128D to 59 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.03 s, search 0.01 s): objective=530.272 imbalance=1.085 nsplit=0       
0/23
1/23
2/23
3/23
4/23
5/23
6/23
7/23
8/23
9/23
10/23
11/23
12/23
13/23
14/23
15/23
16/23
17/23
18/23
19/23
20/23
21/23
22/23
faiss_search
 faiss search time:  0.0004960519727319479
get_result_dic 0.0012823579600080848
Time of Retrieval(On_line):0.961330943973735
1/97
q_feature_num
273 features in  images

Clustering 414 points in 128D to 7 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.02 s, search 0.00 s): objective=118.408 imbalance=1.065 nsplit=0       
0/14
1/14
2/14
3/14
4/14
5/14
6/14
7/14
8/14
9/14
10/14
11/14
12/14
13/14
faiss_search
 faiss search time:  3.552797716110945e-05
get_result_dic 0.012248728075064719
Time of Retrieval(On_line):0.33105905004777014
2/97
q_feature_num
501 features in  images

Clusteri

WARNING clustering 168 points to 9 centroids: please provide at least 351 training points



Clustering 168 points in 128D to 9 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.02 s, search 0.00 s): objective=39.5143 imbalance=1.208 nsplit=0       
0/4
1/4
2/4
3/4
faiss_search
 faiss search time:  2.7860049158334732e-05
get_result_dic 0.00033045304007828236
Time of Retrieval(On_line):0.31396897905506194
4/97
q_feature_num
452 features in  images

Clustering 912 points in 128D to 11 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.01 s, search 0.00 s): objective=246.809 imbalance=1.040 nsplit=0       
0/6
1/6
2/6
3/6
4/6
5/6
faiss_search
 faiss search time:  3.6205979995429516e-05
get_result_dic 0.0003345540026202798
Time of Retrieval(On_line):0.3213462370913476
5/97
q_feature_num
1137 features in  images


WARNING clustering 1006 points to 29 centroids: please provide at least 1131 training points



Clustering 1006 points in 128D to 29 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.01 s, search 0.00 s): objective=224.318 imbalance=1.153 nsplit=0       
0/13
1/13
2/13
3/13
4/13
5/13
6/13
7/13
8/13
9/13
10/13
11/13
12/13
faiss_search
 faiss search time:  0.00011416897177696228
get_result_dic 0.012340727960690856
Time of Retrieval(On_line):0.473720868001692
6/97
q_feature_num
161 features in  images


...

WARNING clustering 285 points to 10 centroids: please provide at least 390 training points



Clustering 285 points in 128D to 10 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.02 s, search 0.00 s): objective=73.8812 imbalance=1.095 nsplit=0       
0/13
1/13
2/13
3/13
4/13
5/13
6/13
7/13
8/13
9/13
10/13
11/13
12/13
faiss_search
 faiss search time:  5.256803706288338e-05
get_result_dic 0.0028332569636404514
Time of Retrieval(On_line):0.33204249397385865
93/97
q_feature_num
545 features in  images

Clustering 544 points in 128D to 13 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.03 s, search 0.00 s): objective=141.023 imbalance=1.172 nsplit=0       
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
faiss_search
 faiss search time:  8.292298298329115e-05
get_result_dic 0.019008915056474507
Time of Retrieval(On_line):0.3835608829976991
94/97
q_feature_num
/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/q_crop_val/60/5_1_False.jpg
extract feature error


WARNING clustering 670 points to 18 centroids: please provide at least 702 training points


get_db_vlad_feat0.02 s, search 0.00 s): objective=171.697 imbalance=1.079 nsplit=0       
0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
faiss_search
 faiss search time:  0.00012065900955349207
get_result_dic 0.0005439340602606535
Time of Retrieval(On_line):0.39277020597364753
95/97
q_feature_num
267 features in  images

Clustering 766 points in 128D to 6 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.02 s, search 0.00 s): objective=206.553 imbalance=1.107 nsplit=0       
0/7
1/7
2/7
3/7
4/7
5/7
6/7
faiss_search
 faiss search time:  2.693803980946541e-05
get_result_dic 0.00869116501417011
Time of Retrieval(On_line):0.32848791196011007
96/97
q_feature_num
419 features in  images

Clustering 232 points in 128D to 10 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
get_db_vlad_feat0.01 s, search 0.00 s): objective=61.2548 imbalance=1.221 nsplit=0       
0/11
1/11
2/11
3/11
4/11

WARNING clustering 232 points to 10 centroids: please provide at least 390 training points


### 2.4 Merge& Validation : merge_val
    - Merge SIFT& VIT 
        1. Cosine-Similarity Matching Score
    - Sort& Filter by topk
    - Validation
        - mAP@topk
            - micro/macro
        - Recall@1
        - Precision@1

In [1]:
#input : path
#if you want to do Validation, turn on val is true
def merge_val(result_path, panorama_id, topk, match_weight=1/4, method='vit', algo='max'):
    
    '''
    topk: int, the number of matching candidates
    match_weight: float, threshold of whether matched or not
    result_path: str, folder path saving results, './data/result/'
    method: str, what you want to use, ['vit', 'sift', 'vit_sift', 'sift_vit']
    algo: str, matching algorithm, ['max', 'erase']
        - 'max' : matching pairs for maximizing score
        - 'erase' : matching pairs removing top1 prediction sequentially.
    val: bool, whether evaluate or not
    '''
    
    #merge
    merge_dict = {}
        
    for m in method.split('_'):
        try:
            with open(f"{result_path}/{m}_best_pair/pair_{panorama_id}_{m}.txt", "r") as f:
                for line in f.readlines():
                    data = line.strip().split(',')
                    img1 = data[0].split('.')[0]
                    img2 = data[1].split('.')[0]
                    score = float(data[2]) #cosine_similarity

                    if img1+'-'+img2 not in merge_dict: #3가지 방법에서 특정 q_db pair가 중복된다면, 점수를 합산!
                        merge_dict[img1+'-'+img2] = score
                    else:
                        merge_dict[img1+'-'+img2] += score
        except:
            print(f"method {m} error")
    
    #change form
    result_dict = {}
    
    for key in merge_dict:
        imgs = key.split('-')
        q = imgs[0]
        db = imgs[1]
        del imgs
        score = merge_dict[key]
        
        #특정 query에 대해, 모든 ref를 list에 모아주기
        if q not in result_dict: #이 query가 처음 나왔을 때
            result_dict[q] = [(db, score)]
        else: #query가 중복될 경우
            db_list = result_dict[q]
            db_list.append((db, score))
            result_dict[q] = db_list
    
    #Matching Algorithm
    if algo == 'max':
    
        result_topk = {}

        for q in result_dict.keys():

            #use match score sorted
            topk_list = sorted(result_dict[q], key=lambda x: -x[1])[:topk] #(db, score)
            result_topk[q] = topk_list
            
    else: #algo == 'erase'
        result_topk = {}
        erase_list = []

        for q in result_dict.keys():

            #erase top1 matching
            erased_list = [(db, score) for (db, score) in result_dict[q] if db not in erase_list]

            if len(erased_list) != 0: #not empty
                #use match score sorted
                topk_list = sorted(erased_list, key=lambda x: -x[1])[:topk] #(db, score)
                result_topk[q] = topk_list

                #add erased top1 matched db, not score
                erase_list.append(topk_list[0][0])
            else: #empty -> score maximization
                topk_list = sorted(result_dict[q], key=lambda x: -x[1])[:topk] #(db, score)
                result_topk[q] = topk_list
    
    
    ############ validation ################
    #ex) query_name : '0_100_False'/ db_name : '2_100_False'
    ########################################
        
    #matched pair의 score
    matched_score = []
    #unmatched pair의 score
    unmatched_score = []

    #panorama mAP : topk 안에 들면 +1/ 아니면 0 -> 파노라마 당 ap 
    pap = 0
    #crop mAP : 간판 크롭 1개 당 ap
    ap = 0

    for q_name in list(result_topk.keys()):
        #query_label
        q_lbl = int(q_name.split('_')[1])

        for rank, (db_name, score) in enumerate(result_topk[q_name], start=1): #only db_name
            #db_label
            db_lbl = int(db_name.split('_')[1])

            if (q_lbl == db_lbl) & (q_lbl != 100): #('False' in q_name) # Count #100 not matched / + except change

                #if matched, record score
                matched_score.append(score)

                #panorama mAP
                pap += 1
                #crop mAP
                qap = 1/rank
                ap += qap
            else: #unmatched

                unmatched_score.append(score)

    #crop mAP
    matched_cnt = len([q_name for q_name in result_topk.keys() if ('100' not in q_name)]) #& ('True' not in q_name)]) ##Divide #except unmatched '100' / + except change

    #panorama mAP
    try:
        pap/=matched_cnt
        print(f"macro AP@{topk} is {pap}") #result_topk 자체가 filter

    except:
        if matched_cnt == 0:
            print(f"there is no gt, pass!") #result_topk 자체가 filter
        else:
            print("stop! - what happen now?")
            
    return result_topk, pap, (ap, matched_cnt), (matched_score, unmatched_score)

In [ ]:
####### hyper params ########

result_path = './data/result/'

topk = 1
method = 'vit
match_weight = 1/4

In [ ]:
### exec

#### variables
#result
final_result_topk = {}

#score
matched_score = []
unmatched_score = []

#panorama mAP
pap_list = []
#crop mAP == recall
cap = 0
matched_cnt = 0

# check time
start = time.perf_counter()

# run merge_val
for i, p_id in enumerate(pnorm_q_dic.keys()): #p_id : panorama_id
    
    #process
    print(f"ID : {p_id} - {i}/{len(pnorm_q_dic)}")
    
    try: # panorama id가 있을 때
        result_topk, pap, cap_tuple, score_tuple = merge_val(result_path=result_path, panorama_id=p_id,\
                                                             topk=topk, method=method)
        print(result_topk)
        final_result_topk[p_id] = result_topk #[(db_name, score), ..., (db_name, score)]
        
        #score thres?
        matched_score += score_tuple[0]
        unmatched_score += score_tuple[1]

        #panorama mAP
        pap_list.append(pap)
        #crop mAP == recall
        cap += cap_tuple[0]
        matched_cnt += cap_tuple[1]

    except: # panorama id가 없을 때
        os.raise.Exception(f"panorama_id {p_id} error")

################ Recall& Precision #####################
#check time
total_time = time.perf_counter()-start
print("total_time", time.perf_counter()-start)
print("query_time", total_time/len(pnorm_q_dic.keys()))

# magic line for distinguish 'matched' from 'unmatched'
match_thres = match_weight*np.mean(matched_score) + (1-match_weight)*np.mean(unmatched_score)

# final result dict
final_dict = {}

# TP : True Positive
# FP : False Positive
tp = 0
fp = 0 

for p_id in list(final_result_topk.keys()): #p_id : panorama_id
    
    thres_result = {}
    
    for q_name in final_result_topk[p_id].keys():
        #query_label
        q_lbl = int(q_name.split('_')[1])
        
        thres_list = []
        
        for rank, (db_name, score) in enumerate(final_result_topk[p_id][q_name], start=1): #only db_name
            #db_label
            db_lbl = int(db_name.split('_')[1])
            
            if score > match_thres: # positve : predict top 1 db
                
                if (q_lbl == db_lbl) & (q_lbl != 100):
                    tp += 1
                else:
                    fp += 1
                    
                thres_list.append(db_name)    
            else: # negative : predict empty list []
                pass
        thres_result[q_name] = thres_list
        
    final_dict[p_id] = thres_result

print(f'macro_mAP@{topk} : {np.mean(pap_list)}')
print(f'micro_mAP@{topk} : {cap/matched_cnt}')
print(f"recall@1:{tp/matched_cnt}") #-> 이건 match_thres를 필터링 하지 않았을 때 top1을 recall로 하고!
print(f"precision@1:{tp/(tp+fp)}") #-> 이건 match_thres 잡아서 precision 잡고!